In [2]:
import numpy as np
from collections import Counter

class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None,*,value=None):
        self.feature = feature # which feature it was devide with
        self.threshold = threshold  # which trashold it was devide with
        self.left = left # left child
        self.right = right # right child
        self.value = value # value if the node is a leaf in the tree
        
    def is_leaf_node(self):
        #if it has value then it is a leaf node
        return self.value is not None
    
    def disp(self):
        print(f"\n\nNode:\nfeature",self.feature, "\t\ntreshold", self.threshold,"\t\nvalue", self.value, "\nleft", self.left, "\nright", self.right, sep='\t')
        if self.left is not None:
            print("LEFT side")
            self.left.disp()
        if self.right is not None:
            print("RIGHT side")
            self.right.disp()


class DecisionTree:
    #max_depth: REQUIRED
    #min_samples_split: REQUIRED
    #min_samples_leaf: REQUIRED
    def __init__(self, max_depth=10, min_samples_split=2, min_samples_leaf = 1, n_features=None, random_state=None, split_criterion="information_gain"):
        self.min_samples_split=min_samples_split # minimum number of samples required to split an internal node
        self.max_depth=max_depth # maximum depth of the tree
        self.n_features=n_features # number of features to consider when looking for the best split, subset of the features is randomly chosen
        self.min_samples_leaf=min_samples_leaf # minimum number of samples required to be at a leaf node
        self.random_state=random_state # random number generator seed for reproducibility
        self.split_criterion = split_criterion # split criterion to use, either "information_gain" or "mse"
        self.root=None

    #For training the model
    def fit(self, X, y):

        # Hiányzó értékek pótlása
        mean = np.nanmean(X, axis=0)  # Oszlopok átlaga (kivéve a hiányzó értékeket)
        inds = np.where(np.isnan(X))  # Hiányzó értékek helyei
        X[inds] = np.take(mean, inds[1])

        self.n_features = X.shape[1] if not self.n_features else min(X.shape[1],self.n_features) # if n_features is not set, then use all the features, else use the min of n_features and X.shape[1]
        self.root = self._grow_tree(X, y)

    def _grow_tree(self, X, y, depth=0):
        #recursively split on the best split
        number_of_samples, number_of_features = X.shape
        number_of_labels = len(np.unique(y)) #if it is 1 then it is a leaf node
        
        # 1.) Check the stopping criteria
        if (depth>=self.max_depth 
            or number_of_labels==1 
            or number_of_samples<self.min_samples_split
            or number_of_samples < self.min_samples_leaf
            ):
            # if any of the above condition is true, then we are at a leaf node
            # so determine the value of the leaf node
            # value is the most common class label
            # return the node
            leaf_value = self._most_common_label(y)
            #print("Kirtérium elérve:",depth)
            return Node(value=leaf_value)
        
        #using random seed for reproducibility if it has been set
        if self.random_state is not None:
            np.random.seed(self.random_state)

        feat_idxs = np.random.choice(number_of_features, self.n_features, replace=False)

        # 2.) Find the best split
        best_feature, best_thresh = self._best_split(X, y, feat_idxs)

        # 3.) Create child nodes
        left_indxes, right_indexs = self._split(X[:, best_feature], best_thresh)
        left = self._grow_tree(X[left_indxes, :], y[left_indxes], depth+1)
        right = self._grow_tree(X[right_indexs, :], y[right_indexs], depth+1)
        return Node(feature=best_feature, threshold=best_thresh, left=left, right=right)

    def _best_split(self, X, y, feat_indexs):
        best_criterion = None
        split_idx, split_threshold = None, None

        for feat_index in feat_indexs:
            X_column = X[:, feat_index]
            thresholds = np.unique(X_column)

            for threshold in thresholds:
                # calculate the information gain
                criterion = self._split_criterion(y, X_column, threshold)

                if best_criterion is None or criterion > best_criterion:
                    best_criterion = criterion
                    split_idx = feat_index
                    split_threshold = threshold

        return split_idx, split_threshold

    def _split_criterion(self, y, X_column, threshold):
        if self.split_criterion == "information_gain":
            return self._information_gain(y, X_column, threshold)
        elif self.split_criterion == "mse":
            return self._mean_squared_error(y, X_column, threshold)
        else:
            raise ValueError("Invalid split criterion: {}".format(self.split_criterion))
        
    def _mean_squared_error(self, y, X_column, threshold):
        left_idxs, right_idxs = self._split(X_column, threshold)
        left_values = y[left_idxs]
        right_values = y[right_idxs]
        left_mean = np.mean(left_values)
        right_mean = np.mean(right_values)
        mse = np.mean((left_values - left_mean) ** 2) + np.mean((right_values - right_mean) ** 2)
        return mse

    def _information_gain(self, y, X_column, threshold):
        # parent entropy
        parent_entropy = self._entropy(y)

        # create children
        left_indexs, right_indexs = self._split(X_column, threshold)

        if len(left_indexs) == 0 or len(right_indexs) == 0:
            return 0
        
        # calculate the weighted avgerage entropy of children
        number_of_samples = len(y)
        number_of_samples_left, number_of_samples_right = len(left_indexs), len(right_indexs)
        entropy_left, entorpy_right = self._entropy(y[left_indexs]), self._entropy(y[right_indexs])
        child_entropy = (number_of_samples_left/number_of_samples) * entropy_left + (number_of_samples_right/number_of_samples) * entorpy_right

        # calculate the information gain
        information_gain = parent_entropy - child_entropy
        return information_gain

    def _entropy(self, y):
        histogram = np.bincount(y) #occurence of each class
        ps = histogram / len(y)
        return -np.sum([p * np.log(p) for p in ps if p>0])

    def _split(self, X_column, split_thresh):
        left_idxs = np.argwhere(X_column <= split_thresh).flatten()
        right_idxs = np.argwhere(X_column > split_thresh).flatten()
        return left_idxs, right_idxs

    def _most_common_label(self, y):
        counter = Counter(y)
        #print("Most common: ", counter.most_common(1))
        value = counter.most_common(1)[0][0]
        return value

    #For predicting the output
    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])

    def _traverse_tree(self, x, node):
        if node.is_leaf_node():
            #print("Levél value: ", node.value, "treshold: ",node.threshold, "feature: ",node.feature)
            return node.value

        if x[node.feature] <= node.threshold:
            #print("Csomopont")
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)
    
    #For printing the tree
    def print(self):
        self.root.disp()
        pass


In [5]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn import tree
import matplotlib.pyplot as plt
import pandas as pd

#Minta adatforrás
#df = pd.read_csv("winequality-red.csv")
#display(df.head())
#X = df[df.columns[:-1]].values
#y = df[df.columns[:-1]].values

wine = datasets.load_breast_cancer()
X = wine.data # adatok
y = wine.target # címkék/célváltozók


# Előkészítsük a bemeneti adatokat és a címkéket
#X = np.array([[1, 2, 3], [4, np.nan, 6], [7, 8, 9]])# Példa bemeneti adatok (a második adat hiányzó értéket tartalmaz)
#y = np.array([0, 1, 0]) # Példa címkék



#Adatok felosztása tanító és tesztelő halmazra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0) # 80% tanító, 20% tesztelő

#Döntési fa tanítása
dt = DecisionTree(max_depth=3, split_criterion="information_gain")
dt.fit(X_train, y_train)

#Döntési fa kiértékelése
y_pred = dt.predict(X_test)

def accuracy(y_test, y_pred):
    return np.sum(y_test == y_pred) / len(y_test)

acc = accuracy(y_test, y_pred)
print("Accuracy: ", acc)

dt.print()

ValueError: object too deep for desired array